In [59]:
from bs4 import BeautifulSoup
import re
import urllib.request
from incapsula import IncapSession, RecaptchaBlocked, MaxRetriesExceeded
import pandas as pd
import os

In [60]:
#url
URL = 'https://raw.githubusercontent.com/cvandeplas/pystemon/master/user-agents.txt'

In [61]:
def get_page(url,page=None):
    import time
    loop = True
    while loop:
        try:
            if(page != None):
                response_c = session.get(url)
            else:
                response_c = session.post(url,page)
            return response_c
        except RecaptchaBlocked as e:
            print('recaptcha blocked')
            time.sleep(5)
        except MaxRetriesExceeded as e:
            print('failed to get page!! retrying...')
            time.sleep(5)
    return False

In [62]:
def get_cats(content, content_type='html'):
    if(content_type == 'html'):
        soup = BeautifulSoup(re.sub("\\\\","", content), 'html5lib')
        cats = soup.find_all('a', {'class':'use-ajax', 'onclick':'lotame_cate_lv1_lv2(this)'})
    elif(content_type == 'json'):
        soup = BeautifulSoup(re.sub("\\'","", pg_c.json()[1]['data']), 'html5lib')
        cats = soup.find_all('a', {'data-types-click':"level_3_category_click"})
        
    if(len(cats) == 0):
        print('categories not found')
    else:
        for i,cat in enumerate(cats):
            print('\t',i,cat.text) 
        while True:
            c = input('choose categorie: ')
            try:
                cat = cats[int(c)]
                return cat
            except:
                pass

In [63]:
def get_name_add(content):
    soup = BeautifulSoup(re.sub("\\\\","", content), 'html5lib')
    
    names = soup.find_all('a',{'class':'normal_title'})
    addresses = soup.find_all('div',{'class':'mapItem'})
    
    adds = [address.text.strip() for address in addresses]
    names = [name.text.strip() for name in names]
    
    return names,adds

In [64]:
def get_n_pages(content):
    soup = BeautifulSoup(re.sub("\\\\","", content), 'html5lib')
    #pag
    pag = soup.select('div.pag > ul > li > a')
    n_pages = 1
    for item in pag:
        n_pages = max(n_pages, int(item['value']))
    return n_pages

In [65]:
r = urllib.request.Request(URL)
r.add_header('Cookie', 'sessionid=13cxrt4uytfc6ijvgeoflmb3u9jmjuhil; csrftoken=jdEKPN8iL62hdaq1hmMuID9DMALiiDIq')
with urllib.request.urlopen(r) as res:
    page = res.read().decode('utf-8')

    print(type(res))

<class 'http.client.HTTPResponse'>


In [84]:
session = IncapSession(user_agent=page.split('\n')[50])

if (os.path.isfile('main_cat.txt')):
    with open('main_cat.txt', 'r') as f:
        pg = f.read()
else:
    URL = 'http://www.yellowpages.com.sg'
    print(page.split('\n')[155])
    pg = get_page(URL).content.decode('utf8')
print(pg[:80])

cat = get_cats(pg)
with open('main_cat.txt','wb') as f:
    f.write(pg.encode('utf-8'))

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version
	 0 Agriculture & Livestock
	 1 Automotive & Transport
	 2 Banking & Finance
	 3 Beauty, Health & Medical
	 4 Business & Professional Services
	 5 Entertainment, Leisure & Hobbies
	 6 Fashion & Textile
	 7 Food & Beverages
	 8 Gifts & Collectibles
	 9 Government & Community
	 10 Home & Office
	 11 Industrial & Engineering
	 12 IT, Electronics & Telecoms
	 13 Marine, Oil & Gas
	 14 Media, Advertising & Printing
	 15 Others
	 16 Packing & Logistics
	 17 Real Estate & Construction
	 18 Safety & Security
	 19 Science, Research & Technology
	 20 Sports & Fitness
	 21 Tourism & Travel
	 22 Training & Education
choose categorie: 8


In [115]:
while True:
    pg_c = get_page(URL+cat['href'])
    if(pg_c.status_code == 400):
        print('You are 100% blacklisted!:(')
    else: 
        cat_c = get_cats(str(pg_c.json()[1]['data']))
        break
# print(URL+cat['href'])pg_c


You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blacklisted!:(
You are 100% blackli

KeyboardInterrupt: 

In [ ]:
pg_c = get_page(URL+cat_c['href'])
cat_c = get_cats(pg_c, content_type='json')

In [ ]:
cat_c

In [ ]:
page = get_page(URL+cat_c['href'])
get_name_add(page.content.decode('utf-8'))

In [ ]:
data = pd.DataFrame(columns=['Company name','Address'])
n_pages = get_n_pages(page.content.decode('utf8'))
for i in range(1,n_pages+1):
    p = get_page('{}{}?page={}'.format(URL,cat_c['href'],i))
    print(n_pages)

    names,adds = get_name_add(p.content.decode('utf-8'))
    g = pd.DataFrame({"Company name":pd.Series(names),"Address":pd.Series(adds)})
    data = data.append(g, ignore_index=True)
    data    

In [ ]:
data.to_csv('{}'.format(cat_c['href'].split('/')[-1]))

In [ ]:
# data.groupby(['Company name'])['Address'].count()
# df.groupby['lang']['value'].sum()

In [ ]:
# g = pd.DataFrame({"Name":pd.Series(['google','fff']),"address":pd.Series(['google','fff'])})
# data.append(g, inplace=True)
# data

In [ ]:
data